In [ ]:
# 📦 Instalar dependências e clonar o repositório DeOldify
!git clone https://github.com/jantic/DeOldify.git DeOldify
%cd DeOldify
!pip install -r requirements-colab.txt
!mkdir 'models'

# ⬇️ Baixar modelo pré-treinado para colorização
!wget https://data.deepai.org/deoldify/ColorizeArtistic_gen.pth -O ./models/ColorizeArtistic_gen.pth

# ⚙️ Imports necessários
from deoldify import device
from deoldify.device_id import DeviceId
device.set(device=DeviceId.GPU0)

import torch
if not torch.cuda.is_available():
    print('⚠️ GPU não está disponível. Vá em "Ambiente de execução > Alterar tipo de hardware > GPU"')

# 🖼️ Colorizador
from deoldify.visualize import *
colorizer = get_image_colorizer(artistic=True)

# 📷 Carregar e colorizar imagem
image_url = 'https://upload.wikimedia.org/wikipedia/commons/9/9a/1920s_Group_Portrait.jpg'

colorizer.plot_transformed_image(
    url=image_url,
    render_factor=35,
    display_render_factor=True,
    figsize=(8,8)
)
